In [3]:
# import dependencies
#import tensorflow as tf
from glob import glob
import pandas as pd
import librosa
import librosa.display
import numpy as np
from librosa.util import fix_length
import scipy as sp
import spafe
from scipy.stats import kurtosis,skew,mode,gstd,describe,iqr,gmean,hmean,median_abs_deviation,variation,tstd,gstd,moment,entropy
from spafe.features import pncc,gfcc,bfcc,mfcc,lfcc,lpc,ngcc,rplp,psrcc,msrcc
from sklearn.preprocessing import normalize

In [16]:
pth=glob('/Users/cecima/Desktop/170B-Proj/Acoustic-Classification/IDMT_Traffic/audio/*') # insert path of folder which contains all recordings

In [22]:
x=pth[0]
print(len(pth))
print(x[74:])

17506
/2019-11-18-07-25_Langewiesener-Strasse_50Kmh_1103360_ME_CH12-BG.wav


In [13]:
# fucntion that extracts features

from spafe.features.gfcc import gfcc # type: ignore
from spafe.features.mfcc import mfcc

def features_extractor(file):
    
    #load the file (audio)
    
    sig, sr = librosa.load(file)
    
    # pad audio files to avoid any dimensional issue
    
    required_audio_size=3
    audio = fix_length(sig, size=required_audio_size*sr) 
    
    #Code for feature extraction

    # local feature extraction (use only one feature as per requirement)
    
    S = gfcc(sig=sig, fs=sr, num_ceps=40,nfilts=128,nfft=2048)
#     S = mfcc(sig=sig, fs=sr, num_ceps=40,nfilts=128,nfft=2048,win_hop=0.0232,win_len=0.0464)
#     S=librosa.feature.melspectrogram(sig, sr=sr, n_mels=128,n_fft=2048,hop_length=512,win_length=1024)

    # Global feature extraction
    
    ft=sp.fft.fft(sig) # code for computing the spectrum using Fast Fourier Transform
    magnitude=np.absolute(ft)
    spec=magnitude[0:11025]
    
    k=kurtosis(spec)
    s=skew(spec)
    mean=np.mean(spec)
    z=np.array(mode(spec)[0])
    mode_var=float(z)
    i=iqr(spec)
    g=gmean(spec)
    h=hmean(spec)
    dev=median_abs_deviation(spec)
    var=variation(spec)
    variance=np.var(spec)
    std=tstd(spec)
    gstd_var=gstd(spec)
    ent= entropy(spec)
    
    features=[mode_var,k,s,mean,i,g,h,dev,var,variance,std,gstd_var,ent]
    features=normalize([features])
    features=np.array(features)
    features=np.reshape(features,(13,))
    feat=[S,features] # save the matrix and vector in a list
    
    return feat

In [14]:
# Function for building the dataframe and calling the feature extractor function to save the features within the dataframe

def preproc_dataset(arr):
    
    features=[]
    for v in arr:
        f=features_extractor(v)
        v = v.replace('.wav', '')
        v=v[74 :] # replace the number (54) in this line with the number of characters to be removed from the beginning of the path of audio file 
        parts = v.split('_')
        parts[2] = parts[2][:-3]
        parts[-1] = parts[-1][2:]
        parts = parts[:6] + [parts[6][0],parts[6][1]] + parts[7:]
        date,location,speed,position,daytime,weather,class_,source_direction,mic,channel= parts
      
        features.append({'file': v,
                           'date': date,
                           'location':location,
                           'speed':speed,
                           'position':position,
                           'daytime':daytime,
                           'weather':weather,
                           'class':class_,
                           'source direction':source_direction,
                          'mic':mic,
                          'channel':channel,
                           'feature': f,
                           })
        
    df_dataset = pd.DataFrame(features, columns=('file','date', 'location','speed','position','daytime','weather','class','source direction','mic','channel','feature'))      
    # df_dataset = pd.DataFrame(mfcc, columns=('file','date', 'location','speed','position','daytime','weather','class','source direction','mic','channel','feature')) 

    return df_dataset

In [15]:
df_train=preproc_dataset(pth)
df_train=df_train.sample(frac=1).reset_index(drop=True)

IndexError: list index out of range

In [ ]:
df_train['speed'].value_counts()

30    2
Name: speed, dtype: int64

In [ ]:
z=df_train['feature']
x=z.tolist()
y=np.array(x,dtype=object)
array=np.save('mvd_gfcc.npy',y)

In [ ]:
df_train=df_train.drop(['feature'], axis=1)

df_train.to_csv('IDMT_gfcc.csv')

In [ ]:
df_train

,file,date,location,speed,position,daytime,weather,class,source direction,mic,channel
0,2019-10-22-08-40_Fraunhofer-IDMT_30Kmh_1141442...,2019-10-22-08-40,Fraunhofer-IDMT,30,1141442,M,D,C,R,ME,12
1,2019-10-22-08-40_Fraunhofer-IDMT_30Kmh_1172589...,2019-10-22-08-40,Fraunhofer-IDMT,30,1172589,M,D,C,L,ME,12


In [ ]:
z[0]

[array([[ 5.25014720e-01, -2.52407914e-01, -8.01031339e-02, ...,
         -6.27609463e-03,  1.69900208e-03, -4.18484708e-04],
        [ 5.14255953e-01, -2.61592034e-01, -6.86822229e-02, ...,
         -4.02687978e-03, -1.00544957e-04, -9.00264144e-04],
        [ 5.25518159e-01, -2.31947959e-01, -9.24935491e-02, ...,
         -5.13212270e-04, -6.27524678e-03, -1.25066628e-03],
        ...,
        [ 4.79450780e-01, -1.74071411e-01, -5.97311582e-02, ...,
         -1.03039598e-03,  2.24265194e-03,  1.65576398e-03],
        [ 5.00892902e-01, -2.01890554e-01, -5.01390147e-02, ...,
         -1.89572607e-03, -4.09183157e-03, -5.07809655e-04],
        [ 5.10684447e-01, -1.69706480e-01, -6.58357976e-02, ...,
         -1.22208986e-03, -3.88642494e-03, -9.96366263e-04]]),
 array([0.00297928, 0.99128825, 0.04585721, 0.00992645, 0.00830653,
        0.00626805, 0.00399821, 0.00333118, 0.00546643, 0.11797512,
        0.01856874, 0.00733199, 0.0256001 ])]